# Specifications - Database
### - Your stakeholder wants you to take the data you have been cleaning and collecting in Parts 1 & 2 of the project and wants you to create a MySQL database for them.

### - Specifically, they want the data from the following files included in your database:
 - Title Basics:
   - Movie ID (tconst)
   - Primary Title
   - Start Year
   - Runtime (in Minutes)
   - Genres
 - Title Ratings
   - Movie ID (tconst)
   - Average Movie Rating
   - Number of Votes
 - The TMDB API Results (multiple files)
   - Movie ID
   - Revenue
   - Budget
   - Certification (MPAA Rating)
- You should normalize the tables as best you can before adding them to your new database.

 - Note: an important exception to their request is that they would like you to keep all of the data from the TMDB API in 1 table together (even though it will not be perfectly normalized).
 - You only need to keep the imdb_id, revenue, budget, and certification columns

### Imports

In [1]:
# Standard Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
# Imports for creating database
import pymysql
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
from urllib.parse import quote_plus as urlquote

# Data Normalization

## Required Transformation Steps for Title Basics:
- Normalize Genre:

 - Convert the single string of genres from title basics into 2 new tables.
  1. title_genres: with the columns:

    - tconst
    - genre_id
  2. genres:

    - genre_id
    - genre_name
- Discard unnecessary information:

 - For the title basics table, drop the following columns:
   - "original_title" (we will use the primary title column instead)
   - "isAdult" ("Adult" will show up in the genres so this is redundant information).
   - "titleType" (every row will be a movie).
   - "genres" and other variants of genre (genre is now represented in the 2 new tables described above.
  - Do not include the title_akas table in your SQL database.
   - You have already filtered out the desired movies using this table, and the remaining data is mostly nulls and not of interest to the stakeholder.

### Load 'Basics' Data

In [2]:
# Load data and datatypes
basics = pd.read_csv('Data/title_basics.csv.gz')
basics.info()
basics.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81900 entries, 0 to 81899
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tconst          81900 non-null  object 
 1   titleType       81900 non-null  object 
 2   primaryTitle    81900 non-null  object 
 3   originalTitle   81900 non-null  object 
 4   isAdult         81900 non-null  int64  
 5   startYear       81900 non-null  float64
 6   endYear         0 non-null      float64
 7   runtimeMinutes  81900 non-null  int64  
 8   genres          81900 non-null  object 
dtypes: float64(2), int64(2), object(5)
memory usage: 5.6+ MB


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
2,tt0068865,movie,Lives of Performers,Lives of Performers,0,2016.0,NaN,90,Drama
3,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
4,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"


## I) Normalizing Genres - Overview
- In order to normalize genres, we will need to:

 - Convert the single string of genres from title basics into 2 new tables.
   1. title_genres: with the columns:

     - tconst
     - genre_id
   2. genres:

     - genre_id
     - genre_name
- Creating these tables will be a multi-step process.

 1. Getting a list of all individual genres.
 2. Create a new title_genres table with with the movie ids duplicated, once for each genre that a movie belongs to.
 3. Create a mapper dictionary with numeric ids for each genre.
 4. Use the mapper dictionary to replace the string genres in title_genres with numeric genre_ids.
 5. Convert the mapper dictionary into a final genres table with the numeric genre_id and the string genre.

### 1. Getting a List of Unique Genres

In [3]:
## Create a col with a list of genres
basics['genres_split'] = basics['genres'].str.split(',')
basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,genres_split
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance","[Comedy, Fantasy, Romance]"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama,[Drama]
2,tt0068865,movie,Lives of Performers,Lives of Performers,0,2016.0,NaN,90,Drama,[Drama]
3,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama,[Drama]
4,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi","[Comedy, Horror, Sci-Fi]"
...,...,...,...,...,...,...,...,...,...,...
81895,tt9914942,movie,Life Without Sara Amat,La vida sense la Sara Amat,0,2019.0,NaN,74,Drama,[Drama]
81896,tt9915872,movie,The Last White Witch,Boku no kanojo wa mahoutsukai,0,2019.0,NaN,97,"Comedy,Drama,Fantasy","[Comedy, Drama, Fantasy]"
81897,tt9916170,movie,The Rehearsal,O Ensaio,0,2019.0,NaN,51,Drama,[Drama]
81898,tt9916190,movie,Safeguard,Safeguard,0,2020.0,NaN,95,"Action,Adventure,Thriller","[Action, Adventure, Thriller]"


In [4]:
# Use .explode() to separate the list of genres into new row
exploded_genres = basics.explode('genres_split')
exploded_genres

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,genres_split
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance",Comedy
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance",Fantasy
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance",Romance
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama,Drama
2,tt0068865,movie,Lives of Performers,Lives of Performers,0,2016.0,NaN,90,Drama,Drama
...,...,...,...,...,...,...,...,...,...,...
81898,tt9916190,movie,Safeguard,Safeguard,0,2020.0,NaN,95,"Action,Adventure,Thriller",Action
81898,tt9916190,movie,Safeguard,Safeguard,0,2020.0,NaN,95,"Action,Adventure,Thriller",Adventure
81898,tt9916190,movie,Safeguard,Safeguard,0,2020.0,NaN,95,"Action,Adventure,Thriller",Thriller
81899,tt9916362,movie,Coven,Akelarre,0,2020.0,NaN,92,"Drama,History",Drama


In [5]:
# use .unique() to get the unique genres from the genres_split column
unique_genres = sorted(exploded_genres['genres_split'].unique())

### 2. Create a new 'title_genres' table


In [6]:
# Save just tconst and genres_split as new df
title_genres = exploded_genres[['tconst', 'genres_split']].copy()
title_genres.head()

,tconst,genres_split
0,tt0035423,Comedy
0,tt0035423,Fantasy
0,tt0035423,Romance
1,tt0062336,Drama
2,tt0068865,Drama


### 3. Create a genre mapper dictionary to replace string genres with integers


In [7]:
## Making the genre mapper dictionary
genre_ints = range(len(unique_genres))
genre_map = dict(zip(unique_genres, genre_ints))
genre_map

{'Action': 0,
 'Adult': 1,
 'Adventure': 2,
 'Animation': 3,
 'Biography': 4,
 'Comedy': 5,
 'Crime': 6,
 'Drama': 7,
 'Family': 8,
 'Fantasy': 9,
 'Game-Show': 10,
 'History': 11,
 'Horror': 12,
 'Music': 13,
 'Musical': 14,
 'Mystery': 15,
 'News': 16,
 'Reality-TV': 17,
 'Romance': 18,
 'Sci-Fi': 19,
 'Sport': 20,
 'Talk-Show': 21,
 'Thriller': 22,
 'War': 23,
 'Western': 24}

### 4. Replace the string genres in 'title_genres' with the new integer ids.


In [8]:
## make new integer genre_id and drop string genres
basics['genre_id'] = basics['genres_split'].replace(genre_map)
basics = basics.drop(columns='genres_split')

### 5. Convert the genre map dictionary into a dataframe.


In [9]:
# Using pd.DataFrame and a dictionary
genres = pd.DataFrame({'Genre_Name': genre_map.keys(),
                         'Genre_ID':genre_map.values()})
genres.head()

,Genre_Name,Genre_ID
0,Action,0
1,Adult,1
2,Adventure,2
3,Animation,3
4,Biography,4


### Discard unnecessary infomation:

In [10]:
# Drop all unnecessary columns
basics = basics.drop(columns=['titleType', 'originalTitle', 'isAdult', 
                    'endYear', 'genres', 'genre_id'])
# Final dataframe check
basics.head()

,tconst,primaryTitle,startYear,runtimeMinutes
0,tt0035423,Kate & Leopold,2001.0,118
1,tt0062336,The Tango of the Widower and Its Distorting Mi...,2020.0,70
2,tt0068865,Lives of Performers,2016.0,90
3,tt0069049,The Other Side of the Wind,2018.0,122
4,tt0088751,The Naked Monster,2005.0,100


## Title Ratings

### Load the Data

In [11]:
# Check to see if all required columns are present
ratings = pd.read_csv('Data/title_ratings.csv.gz')
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1989
1,tt0000002,5.8,265
2,tt0000005,6.2,2635
3,tt0000006,5.0,183
4,tt0000007,5.4,825


## TMDB API Results

In [12]:
# Load TMDB Dataset
tmdb = pd.read_csv('Data/tmdb_results_combined.csv.gz')
tmdb.info()
tmdb.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2596 entries, 0 to 2595
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   imdb_id                2596 non-null   object 
 1   adult                  2594 non-null   float64
 2   backdrop_path          1441 non-null   object 
 3   belongs_to_collection  211 non-null    object 
 4   budget                 2594 non-null   float64
 5   genres                 2594 non-null   object 
 6   homepage               170 non-null    object 
 7   id                     2594 non-null   float64
 8   original_language      2594 non-null   object 
 9   original_title         2594 non-null   object 
 10  overview               2551 non-null   object 
 11  popularity             2594 non-null   float64
 12  poster_path            2353 non-null   object 
 13  production_companies   2594 non-null   object 
 14  production_countries   2594 non-null   object 
 15  rele

,imdb_id,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tt0113026,0.0,/vMFs7nw6P0bIV1jDsQpxAieAVnH.jpg,NaN,10000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 10402, '...",NaN,62127.0,en,The Fantasticks,...,0.0,86.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Try to remember the first time magic happened,The Fantasticks,0.0,5.50,22.0,NaN
2,tt0113092,0.0,NaN,NaN,0.0,"[{'id': 878, 'name': 'Science Fiction'}]",NaN,110977.0,en,For the Cause,...,0.0,100.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,The ultimate showdown on a forbidden planet.,For the Cause,0.0,5.45,10.0,NaN
3,tt0116391,0.0,NaN,NaN,0.0,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...",NaN,442869.0,hi,Gang,...,0.0,152.0,"[{'english_name': 'Hindi', 'iso_639_1': 'hi', ...",Released,NaN,Gang,0.0,4.00,1.0,NaN
4,tt0118694,0.0,/n4GJFGzsc7NinI1VeGDXIcQjtU2.jpg,NaN,150000.0,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",NaN,843.0,cn,花樣年華,...,14204632.0,99.0,"[{'english_name': 'Cantonese', 'iso_639_1': 'c...",Released,"Feel the heat, keep the feeling burning, let t...",In the Mood for Love,0.0,8.10,2298.0,PG


In [13]:
# Drop all unnecessary columns
tmdb = tmdb.drop(columns=['adult', 'backdrop_path', 'belongs_to_collection', 'homepage', 'genres', 'id',
                         'original_language', 'original_title', 'overview', 'popularity', 'poster_path',
                         'production_companies', 'production_countries', 'release_date', 'runtime',
                         'spoken_languages', 'status', 'tagline', 'title', 'video', 'vote_average', 'vote_count'])
# Final dataframe check
tmdb.head()

,imdb_id,budget,revenue,certification
0,0,NaN,NaN,NaN
1,tt0113026,10000000.0,0.0,NaN
2,tt0113092,0.0,0.0,NaN
3,tt0116391,0.0,0.0,NaN
4,tt0118694,150000.0,14204632.0,PG


# MySQL Database Requirements
- Use sqlalchemy with Pandas to execute your SQL queries inside your notebook.

- Create a new database on your MySQL server and call it "movies."

- Make sure to have the following tables in your "movies" database:

 - title_basics
 - title_ratings
 - title_genres
 - genres
 - tmdb_data
- Make sure to set a Primary Key for each table that isn't a joiner table (e.g., title_genres is a joiner table).

- After creating each table, show the first 5 rows of that table using a SQL query.

- Make sure to run the "SHOW TABLES" SQL query at the end of your notebook to show that all required tables have been created.